In [6]:
# imports
import pandas as pd
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
from torch.utils.data import Dataset
from datasets import load_dataset

## Load Emotions Dataset

In [8]:
emo_train = load_dataset("csv", data_files="./Swahili Emotion Data/emo_train.csv", encoding = "ISO-8859-1")
emo_valid = load_dataset("csv", data_files="./Swahili Emotion Data/emo_valid.csv", encoding = "ISO-8859-1")
emo_test = load_dataset("csv", data_files="./Swahili Emotion Data/emo_test.csv", encoding = "ISO-8859-1")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
# emo_train_df = pd.read_csv("./Swahili Emotion Data/emo_train.csv", encoding = "ISO-8859-1")
# display(emo_train_df)

# emo_valid_df = pd.read_csv("./Swahili Emotion Data/emo_valid.csv", encoding = "ISO-8859-1")
# display(emo_valid_df)

# emo_test_df = pd.read_csv("./Swahili Emotion Data/emo_test.csv", encoding = "ISO-8859-1")
# display(emo_test_df)

,text,labels
0,Ni zaidi ya Asha ngedere,[4]
1,Tuendelee kuchapa kazi ndugu zangu tena kwa fu...,[1]
2,waziri anasoma utadhani uamefungwa mashine,[0]
3,Ukweli ni kwamba mjinga ndie ambae haoni Magu...,"[4, 5]"
4,Hapo mlijiridhisha nn sasa??,[5]
...,...,...
9727,"Tunaomba muongeze siku za mkopo kidogo, wanafu...",[3]
9728,hukupaswa kulimbikiza vyombo vichafu kwenye si...,[4]
9729,"Pole sana wafiwa. Pole mama Janet, pole sana m...",[3]
9730,Ninahisi kuharibiwa na ni mbaya zaidi kwa mama...,[3]


,text,labels
0,Hee makubwa?? wanasema washa zoea mabomu??,[5]
1,"Hakuna anayekataa foreign investors, tatizo hi...",[4]
2,Utazidi kufanikiwa kwa sababu hukuwasahau waza...,[1]
3,Mbaya?,[0]
4,Cha kuchekesha hapa ni kipi sasa! Watu walisha...,[5]
...,...,...
1292,Kunywa maji upunguze presha,[0]
1293,Sio vyema jamani uchungu wa mtoto kisha nakuib...,[3]
1294,Afadhali yako. Ni vitu gani umewekea jitihada ...,[1]
1295,Baadhi ya wanafunzi walishindwa kustahimili ma...,[3]


,text,labels
0,"Sasa walipokuja, walikuja kama nani? wapiga ch...",[5]
1,Waziri wa kilimo na naibu wake wapo vizuri san...,[1]
2,"Akajielekeza, hisia zake za uchungu zikimujaa.",[3]
3,Yakobo alijawa na huzuni kwa yule mwanamke amb...,[3]
4,nimepata maumivu ya kichwa na kikohozi .,[3]
...,...,...
1942,"""Ah ni ya kuchekesha!"" Alisema Hidaya, naye a...",[1]
1943,"Mama yao alikufa mwezi mmoja baadaye, aliumia.",[3]
1944,"Watu walianza kupukutika kwa kasi,, aibu na ha...","[3, 6]"
1945,ni unamaanisha nini?nimenunuwa kompyuta hii mi...,[5]


## Load Pre-Trained Model
### AfriBerta

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("castorini/afriberta_base")
model = AutoModelForMaskedLM.from_pretrained("castorini/afriberta_base")

tokenizer_config.json:   0%|          | 0.00/257 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.55M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/446M [00:00<?, ?B/s]

## Preprocess Data

In [ ]:
import torch
from torch.utils.data import Dataset




In [9]:
# Preprocessing the Emotion Dataset using the Trainer

def tokenize_function(data):
    return tokenizer(data["text"], padding = "max_length", truncation=True)

tokenised_trained_emotion = emo_train.map(tokenize_function, batched=True)
tokenised_valid_emotion = emo_valid.map(tokenize_function, batched=True)
tokenised_test_emotion = emo_test.map(tokenize_function, batched=True)


Map:   0%|          | 0/9732 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1297 [00:00<?, ? examples/s]

Map:   0%|          | 0/1947 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

import evaluate

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenised_trained_emotion,
    eval_dataset=tokenised_valid_emotion,
    compute_metrics=compute_metrics,
)
trainer.train()

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`